In [1]:
from tqdm import tqdm
from platform import python_version
import pandas as pd
import numpy as np
import csv
from pathlib import Path
import os

In [2]:
print(python_version())

3.10.5


In [3]:
df = pd.read_csv("node.csv")

In [4]:
id = df['id'].tolist()

In [5]:
df_2 = pd.read_csv("rel.csv")
st = df_2['start'].tolist()
en = df_2['end'].tolist()
ef = df_2['effect'].tolist()
im = df_2['impact'].tolist()

In [6]:
dfc = pd.read_csv("newUn.csv", sep='\t', skip_blank_lines=False)
dfim = pd.read_csv("newim.csv", sep='\t', skip_blank_lines=False)
dfef = pd.read_csv("newefc.csv", sep='\t', skip_blank_lines=False)
dfc.head()

,ids
0,"1,10,89,53,55,141,149,173,176,28,16,330,189,56..."
1,"273,16,176,674,812,29,422,1310,1378,186,1469,5..."
2,"1,16,55,100,58,161,56,72,8,10,33,6,422,49,11,1..."
3,"1,42,87,161,58,618,6,8,9,1272,2528,83,235,6702..."
4,"1,16,58,56,6,29,53,186,383,106,8,234,127,129,2..."


In [7]:
con = dfc['ids'].tolist()
imp = dfim['ids'].tolist()
eff = dfef['ids'].tolist()
print(len(con))

165465


In [8]:
conSet = []
impSet = []
effSet = []
for x in range(len(con)):
    tb = []
    tc = []
    td = []
    if con[x] is np.nan:
        conSet.append(tb)
        impSet.append(tc)
        effSet.append(td)
    else:
        tb = list(map(int, con[x].split(',')))
        tc = list(map(float, str(imp[x]).split(',')))
        td = list(map(int, eff[x].split(',')))
        conSet.append(tb)
        impSet.append(tc)
        effSet.append(td)
print(len(conSet[1]), ' ', len(impSet[1]), ' ', len(effSet[1]))

261   261   261


In [ ]:
conSets = []
impSets = []
effSets = []
cnt = 0
for x in range(len(conSet)):
  tx = []
  ty = []
  tz = []
  for y in range(len(conSet[x])):
    d = conSet[x][y]
    flag = 0
    if d < 165465:
      for z in range(len(conSet[d])):
        if conSet[d][z] == x:
          flag = 1
          cnt += 1
    if flag == 0:
      if conSet[x][y] < 165465:
        tx.append(conSet[x][y])
        ty.append(impSet[x][y])
        tz.append(effSet[x][y])
  conSets.append(tx)
  impSets.append(ty)
  effSets.append(tz)
conSet = conSets
impSet = impSets
effSet = effSets
print(cnt)

In [9]:
def getlist(q, p, r, s):
  so = []
  params = []
  np = []
  su = []
  #print(p)
  chc = conSet[q]
  ch = conSet[p]
  ech = effSet[p]
  for y in range(len(ch)):
    flag = 0
    mod = 0
    end = ch[y]
    efx = ech[y]
    if efx == 'positive':
      mod = 1*r
    else:
      mod = -1*r
    for z in range(len(chc)):
      if chc[z] == end:
        flag = 1
    if flag ==0:
      so.append(q)
      params.append(end)
      np.append(mod)
      su.append(s)
  return so, params, np, su

In [10]:
startd = []
endd = []
effectd = []
impactd = []
for x in tqdm(range(len(con))):
  start = x
  param = conSet[x]
  if len(param) > 0:
    efx = effSet[x]
    efp = impSet[x]
    impact = efp[0]
    while len(param) != 0:
      src = []
      nen = []
      nenp = []
      impx = []
      for z in range(len(param)):
        modif = efp[z]
        src, nen, nenp, impx = getlist(start, param[z], modif, impact)
      for n in range(len(nenp)):
        if nenp[n] == 1:
          nenp[n] = 'positive'
        else:
          nenp[n] = 'negative'
      for m in range(len(nen)):
        startd.append(src[m])
        endd.append(nen[m])
        effectd.append(nenp[m])
        impactd.append(impx[m])
      param = nen
      efp = nenp
      #print(param)

  0%|          | 100/165465 [03:57<109:07:14,  2.38s/it]


KeyboardInterrupt: 

In [ ]:
list_tuples = list(zip(startd, endd, effectd, impactd))  
d = pd.DataFrame(list_tuples, columns=['start', 'end', 'effect', 'impact'])  

In [ ]:
stx = d['start'].tolist()
enx = d['end'].tolist()
efx = d['effect'].tolist()
imx = d['impact'].tolist()
idb = d.index.values.tolist()

In [ ]:

srcy = []
neny = []
nenpy = []
impxy = []
def worker():
  cnt = 0
  for x in tqdm(range(len(stx))):
    srcd = stx[x]
    endc = enx[x]
    efc = efx[x]
    impc = imx[x]
    idg = idb[x]
    flag = 0
    for y in range(len(stx)):
      s = stx[y]
      e = enx[y]
      ec = efx[y]
      i = imx[y]
      id = idb[y]
      if idg != id:
        if srcd == s and endc == e:
          ch1 = abs(float(impc))
          ch2 = abs(float(i))
          if ch1 < ch2:
            flag = 1
            cnt += 1
    if flag != 1:
      srcy.append(srcd)
      neny.append(endc)
      nenpy.append(efc)
      impxy.append(impc)

In [ ]:
for x in tqdm(range(len(stx))):
  p = len(stx)
  worker(x, p)

In [ ]:
print('Report: removed ',cnt,' entries')
list_tuple = list(zip(srcy, neny, nenpy))  
dd = pd.DataFrame(list_tuple, columns=['start', 'end', 'effect'])  
dd.drop_duplicates(keep='first', inplace=True)
dd.to_csv(r'ext.csv', index=False)
print('File written as ext.csv')